In [26]:
import numpy as np
import pandas as pd

In [ ]:
# Now let's check if the index is correctly set when reading the saved file
path = '../stocks_data/AAPL_03783310_14593.csv'
df = pd.read_csv(path, index_col='datetime', parse_dates=True)  # Parse datetime and set as index
df['stock_exret']

In [ ]:
cols = ['prc', 'target', 'log_diff', 'stock_exret', 'rf', 'adj_price', 'total_return']
df[cols]

In [29]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df

In [ ]:
df.index[0]

In [ ]:
# Define the ticker symbol and date range
ticker = 'AAPL'
start_date = df.index[0]
end_date = df.index[-1]

# Download the data
data = yf.download(ticker, start=start_date, end=end_date)
data['Close']


In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot data['Adj Close'] on the left y-axis
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Adj Close (Yahoo Finance)', color=color)
ax1.plot(data.index, data['Adj Close'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a second y-axis for df['adj_price']
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Adj Price (Your Method)', color=color)
ax2.plot(df.index, df['adj_price'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Comparison of Adjusted Prices of AAPL on Separate Y-Axes')
fig.tight_layout()
plt.show()


In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot data['Adj Close'] on the left y-axis
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Adj Close (Yahoo Finance)', color=color)
ax1.plot(df.index, df['prc'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a second y-axis for df['adj_price']
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Adj Price (Your Method)', color=color)
ax2.plot(data.index, data['Close'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Comparison of Adjusted Prices of AAPL on Separate Y-Axes')
fig.tight_layout()
plt.show()

In [ ]:
# load t1 from objects/t1.pkl
t1 = pd.read_pickle('../objects/t1.pkl')
t1

# Applying notes ch4 AFML on appl

In [ ]:
t1

In [37]:
def mpNumCoEvents(closeIdx, t1, molecule):
    '''
    Compute the number of concurrent events per bar.
    +molecule[0] is the date of the first event on which the weight will be computed
    +molecule[-1] is the date of the last event on which the weight will be computed
    Any event that starts before t1[molecule].max() impacts the count.
    '''
    #1) find events that span the period [molecule[0], molecule[-1]]
    t1 = t1.fillna(closeIdx[-1])  # unclosed events still must impact other weights
    t1 = t1[t1 >= molecule[0]]  # events that end at or after molecule[0]
    t1 = t1.loc[:t1[molecule].max()]  # events that start at or before t1[molecule].max()
    
    #2) count events spanning a bar
    iloc = closeIdx.searchsorted(np.array([t1.index[0], t1.max()]))
    count = pd.Series(0, index=closeIdx[iloc[0]:iloc[1] + 1])
    for tIn, tOut in t1.items():  # Changed from iteritems() to items()
        count.loc[tIn:tOut] += 1
    return count.loc[molecule[0]:t1[molecule].max()]

In [ ]:
df

In [ ]:
close = df['adj_price']
close

In [ ]:
numCoEvents = mpNumCoEvents(closeIdx = df.index, t1 = t1, molecule = t1.index)
numCoEvents = numCoEvents.loc[~numCoEvents.index.duplicated(keep="last")]
numCoEvents=numCoEvents.reindex(close.index).ﬁllna(0)
numCoEvents

In [41]:
def mpSampleTW(t1, numCoEvents, molecule):
    # Derive average uniqueness over the event's lifespan
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():  # Changed from .iteritems() to .items()
        wght.loc[tIn] = (1. / numCoEvents.loc[tIn:tOut]).mean()
    return wght

In [ ]:
mpSampleTW(t1 = t1, numCoEvents = numCoEvents, molecule = t1.index).mean()

In [43]:
# **SNIPPET 4.10** **DETERMINATION OF SAMPLE WEIGHT BY ABSOLUTE RETURN ATTRIBUTION**
def mpSampleW(t1, numCoEvents, close, molecule):
    # Derive sample weight by return attribution
    ret = np.log(close).diff()  # log-returns, so that they are additive
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():  # Changed from iteritems() to items()
        wght.loc[tIn] = (ret.loc[tIn:tOut] / numCoEvents.loc[tIn:tOut]).sum()
    return wght.abs()

In [ ]:
mpSampleW(t1 = t1, numCoEvents = numCoEvents, close = close, molecule = t1.index)

# Fracti diff

In [ ]:
# plot adj_price and stock_exret

import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('time')
ax1.set_ylabel('price', color=color)
ax1.plot(df.index, df['frac_diff'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('stock_exret', color=color)
ax2.plot(df.index, df['adj_price'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

In [ ]:
# SNIPPET 5.3 THE NEW FIXED-WIDTH WINDOW FRACDIFF METHOD
def fracDiff_FFD(series, d, thres=1e-5):
    '''
    Constant width window (new solution)
    Note 1: thres determines the cut-off weight for the window
    Note 2: d can be any positive fractional, not necessarily bounded [0,1].
    '''
    #1) Compute weights for the longest series
    w = getWeights_FFD(d, thres)
    width = len(w) - 1
    #2) Apply weights to values
    df = {}
    for name in series.columns:
        # Corrected the fill method to 'ffill'
        seriesF, df_ = series[[name]].fillna(method='ffill').dropna(), pd.Series()
        for iloc1 in range(width, seriesF.shape[0]):
            loc0, loc1 = seriesF.index[iloc1-width], seriesF.index[iloc1]
            if not np.isfinite(series.loc[loc1, name]):
                continue  # exclude NAs
            df_[loc1] = np.dot(w.T, seriesF.loc[loc0:loc1])[0, 0]
        df[name] = df_.copy(deep=True)
    df = pd.concat(df, axis=1)
    return df


def getWeights_FFD(d, thres):
    # Appends while the last weight is above the threshold
    w = [1.]
    while abs(w[-1]) > thres:
        w_ = -w[-1] / len(w) * (d - len(w) + 1)
        w.append(w_)
    w = np.array(w[::-1]).reshape(-1, 1)
    return w

fracDiff_FFD(df[['adj_price']], d=0.5, thres=0.01)

In [ ]:
def plotMinFFD(df, col_name = 'adj_price'):
    from statsmodels.tsa.stattools import adfuller
    import matplotlib.pyplot as plt
    d_found = False
    
    out = pd.DataFrame(columns=['adfStat', 'pVal', 'lags', 'nObs', '95% conf', 'corr'])
    
    # Assuming 'Close' is the column name in df
    for d in np.linspace(0, 1, 21):
        df1 = np.log(df[[col_name]])# .resample('1D').last()  # Downcast to daily observations
        df2 = fracDiff_FFD(df1, d, thres=.01)
        corr = np.corrcoef(df1.loc[df2.index, col_name], df2[col_name])[0, 1]
        adf_result = adfuller(df2[col_name], maxlag=1, regression='c', autolag=None)
        out.loc[d] = list(adf_result[:4]) + [adf_result[4]['5%']] + [corr]  # With critical value
        print(f'Fractional differentiation order: {d}, ADF Statistic: {adf_result[0]}, Correlation: {corr}')
        if not d_found and adf_result[1] < 0.05:
            d_found = True
            d_req = d
            print(f'Fractional differentiation order found: {d_req}')

    # Plotting the results
    out[['adfStat', 'corr']].plot(secondary_y='adfStat')
    plt.axhline(out['95% conf'].mean(), linewidth=1, color='r', linestyle='dotted')
    plt.title('ADF Statistic and Correlation vs. Differentiation Order')
    plt.show()
    
    return out, d_req

# Assuming 'df' is the DataFrame in the local environment
result, d = plotMinFFD(df)

In [ ]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('time')
ax1.set_ylabel('price', color=color)
ax1.plot(df.index, df['adj_price'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Fractioned Diff', color=color)
col = 'log_price'
frac_diff = fracDiff_FFD(df[[col]], d=d, thres=0.01)
ax2.plot(frac_diff.index, frac_diff, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

In [ ]:
df[['log_price']]

In [ ]:
frac_diff

In [ ]:
df_ = df.copy()
df_['frac'] = frac_diff
df_['frac']=df_['frac'].fillna(0)
df_